In [ ]:
# Import Libraries
import pandas as pd
import numpy as np

# Read data
path = '01 - Data/ecom customer_data.xlsx'

In [ ]:
df = pd.read_excel(path)

In [ ]:
no_gender = df.copy()
no_gender.dropna(subset = ['Gender'], inplace = True)
no_gender['Gender'] = no_gender['Gender'].fillna('No Gender')
no_gender.head(10)